# Control variates, antithetic variates, QMC


Consider $X ∼ N_{d}(0, I_{d})$ (standard Gaussian distribution of dimension d). \
We want to compute the probability that $X ∈ A$ for a certain $A ⊂ R^2$. We consider for example:

$A = \{(x_{1}, . . . , x_{d}), \mathopen{|} \prod_{i=1}^n x_{i}\mathclose{|} ≤ c\}$


In [5]:
import numpy as np

### Approximation of the probability

In [9]:
def gaussian_probability(d, N, c):
    X = np.random.multivariate_normal(np.zeros(d), np.identity(d), size = N)
    product = np.abs(np.prod(X, axis=1))
    return np.mean(product <= c)

In [15]:
d=5
N=100
c=.1
gaussian_probability(d, N, c)

0.6

### Control variates

In [23]:
def gaussian_probability_cv(d, N, c, beta):
    X = np.random.multivariate_normal(np.zeros(d), np.identity(d), size = N)
    product = np.abs(np.prod(X, axis=1))
    return np.mean((product <= c) - beta*X[:, 0])

In [70]:
d=5
N=100
c=.0001
beta=.01
gaussian_probability_cv(d, N, c, 1)

-0.08160918626205124

Issue : we can get negative probabilities. Control variates does not work here.

### Antithetic variates

In [19]:
def gaussian_probability_antithetic(d, N, c):
    X = np.random.multivariate_normal(np.zeros(d), np.identity(d), size = N)
    prod = np.abs(np.prod(X, axis=1))
    minus_prod = np.abs(np.prod(-X, axis=1))
    return (np.sum(prod <= c) + np.sum(minus_prod <= c))/(2*N)

In [20]:
d=5
N=100
c=.1
gaussian_probability_antithetic(d, N, c)

0.62

### Variance comparison

In [51]:
Nrep = 1000
probs = np.zeros([3, Nrep])
for i in range(Nrep):
    probs[0, i] = gaussian_probability(d, N ,c)
    probs[1, i] = gaussian_probability_antithetic(d, N ,c)
    probs[2, i] = gaussian_probability_cv(d, N ,c, beta)

In [71]:
np.var(probs, axis=1)

array([0.00071344, 0.00073899, 0.00068398])

### QMC (quasi-Monte Carlo)